['DACON 자율주행 센서의 안테나 성능 예측 AI 경진대회' 원본 코드) [Private_39위] lightgbm+random forest+xgboost](https://dacon.io/competitions/official/235927/codeshare/6651?page=1&dtype=recent)

In [12]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sys
import seaborn as sns
import random as rn
import os
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import GridSearchCV, cross_val_score, RepeatedKFold
from sklearn import metrics

from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.multioutput import MultiOutputRegressor

from collections import Counter
from flaml import AutoML

import warnings
%matplotlib inline
warnings.filterwarnings(action='ignore')
# %cd '/content/drive/MyDrive/Autonomous_driving_antenna/'

In [ ]:
# reproducibility
def set_seed(seed_num):
    # tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    rn.seed(seed_num)
    os.environ['PYTHONHASHSEED']=str(seed_num)

seed_num = 42
set_seed(seed_num)

In [ ]:
train = pd.read_csv('dataset/train.csv')
train_x_df = train.filter(regex='X')   # input
train_y_df = train.filter(regex='Y')   # output

print(train_x_df.shape, train_y_df.shape)
print(train_x_df.isnull().sum())
print(train_y_df.isnull().sum())

In [ ]:

train_x = np.array(train_x_df)
train_y = np.array(train_y_df)

print(train_x.shape, train_y.shape)

# MODEL_TIME_BUDGET = 60*5
# MODEL_METRIC = 'rmse'
# MODEL_TASK = "regression"
# MODEL_LIST = ["lgbm"]

# params = {
#     "time_budget": MODEL_TIME_BUDGET,  
#     "metric": MODEL_METRIC,
#     "estimator_list": MODEL_LIST, 
#     "task": MODEL_TASK,
#     "seed":seed_num,
# }

# auto_lgbm = MultiOutputRegressor(AutoML(**params))
# auto_lgbm.fit(train_x, train_y)

# MODEL_TIME_BUDGET = 60*5
# MODEL_METRIC = 'rmse'
# MODEL_TASK = "regression"
# MODEL_LIST = ["xgboost"]

# params = {
#     "time_budget": MODEL_TIME_BUDGET,  
#     "metric": MODEL_METRIC,
#     "estimator_list": MODEL_LIST, 
#     "task": MODEL_TASK,
#     "seed":seed_num,
# }

# auto_xgb = MultiOutputRegressor(AutoML(**params))
# auto_xgb.fit(train_x, train_y)

# MODEL_TIME_BUDGET = 60*5
# MODEL_METRIC = 'rmse'
# MODEL_TASK = "regression"
# MODEL_LIST = ["rf"]

# params = {
#     "time_budget": MODEL_TIME_BUDGET,  
#     "metric": MODEL_METRIC,
#     "estimator_list": MODEL_LIST, 
#     "task": MODEL_TASK,
#     "seed":seed_num,
# }

# auto_rf = MultiOutputRegressor(AutoML(**params))
# auto_rf.fit(train_x, train_y)

lightgbm = MultiOutputRegressor(LGBMRegressor(colsample_bytree=0.5263478696871312,
              learning_rate=0.01522625637146587, max_bin=1023,
              min_child_samples=23, n_estimators=2950, num_leaves=434,
              reg_alpha=0.013442380441791843, reg_lambda=0.0440062359548784, seed = seed_num,
              verbose=-1))

lightgbm.fit(train_x, train_y)

xgboost = MultiOutputRegressor(XGBRegressor(colsample_bylevel=0.4192302659044743,
             colsample_bytree=0.7180716573534248, grow_policy='lossguide',
             learning_rate=0.013198731967539607, max_depth=0, max_leaves=140,
             min_child_weight=35.072279762160896, n_estimators=803, n_jobs=-1,
             reg_alpha=0.0009765625, reg_lambda=0.6342055478585719,
             subsample=0.827351358517848, tree_method='hist', seed = seed_num,
             use_label_encoder=False, verbosity=0))

xgboost.fit(train_x, train_y)

rf = MultiOutputRegressor(RandomForestRegressor(max_features=0.6347607006852164, max_leaf_nodes=1608,
                      n_estimators=136, n_jobs=-1, random_state = seed_num))

rf.fit(train_x, train_y)

test_df = pd.read_csv('dataset/test.csv')

print(test_df.shape)
test_x = np.array(test_df.drop(columns=['ID']))

lgb_p = lightgbm.predict(test_x)
xgb_p = xgboost.predict(test_x)
rf_p = rf.predict(test_x)

prediction = (lgb_p + xgb_p  + rf_p)/3
print(np.shape(prediction))

submit = pd.read_csv('dataset/sample_submission.csv')
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = prediction[:,idx-1]

submit.to_csv('submissions.csv', index=False)    